In [1]:
!pip install transformers


     |████████████████████████████████| 2.6 MB 7.5 MB/s 
     |████████████████████████████████| 895 kB 38.4 MB/s 
     |████████████████████████████████| 636 kB 45.1 MB/s 
     |████████████████████████████████| 3.3 MB 46.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
pip install -q tf-models-official==2.3.0


     |████████████████████████████████| 840 kB 7.4 MB/s 
     |████████████████████████████████| 211 kB 58.2 MB/s 
     |████████████████████████████████| 37.1 MB 49 kB/s 
     |████████████████████████████████| 679 kB 17.7 MB/s 
     |████████████████████████████████| 352 kB 59.5 MB/s 
     |████████████████████████████████| 99 kB 11.3 MB/s 
     |████████████████████████████████| 1.2 MB 42.2 MB/s 


In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
from official.modeling import tf_utils
from official import nlp
from official.nlp import bert
from sklearn.model_selection import train_test_split
import json
# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks
import pandas as pd
import transformers as ppb
from nltk.corpus import wordnet

In [6]:
with open("training.en-en.data", "r",encoding='UTF-8') as read_file:
    data = json.load(read_file)
with open("training.en-en.gold", "r",encoding='UTF-8') as read_file:
    gold = json.load(read_file)
with open("dev.en-en.data", "r",encoding='UTF-8') as read_file:
    dev_data = json.load(read_file)
with open("dev.en-en.gold", "r",encoding='UTF-8') as read_file:
    dev_gold = json.load(read_file)
with open("test.en-en.data", "r",encoding='UTF-8') as read_file:
    test_data = json.load(read_file)
with open("test.en-en.gold", "r",encoding='UTF-8') as read_file:
    test_gold = json.load(read_file)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
labels_tmp= np.array([dat['tag'] for dat in gold])
labels= [0 if x=='F' else 1 for x in labels_tmp]
labels=np.asarray(labels)

In [8]:
labels_tmp_val= np.array([dat['tag'] for dat in dev_gold])
labels_val= [0 if x=='F' else 1 for x in labels_tmp_val]
labels_val=np.asarray(labels_val)

In [9]:
labels_tmp_test= np.array([dat['tag'] for dat in test_gold])
labels_test= [0 if x=='F' else 1 for x in labels_tmp_test]
labels_test=np.asarray(labels_test)

In [10]:
hub_url_bert = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2"
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/keras_bert/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)

# model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
# tokenizer =tokenizer_class.from_pretrained(pretrained_weights)
tokenizer = bert.tokenization.FullTokenizer(
    vocab_file=os.path.join(gs_folder_bert, "vocab.txt"),
     do_lower_case=True)

print("Vocab size:", len(tokenizer.vocab))
def merge(lst1, lst2):
    return [a + b[1:] for (a, b) in zip(lst1, lst2)]

def encode_sentence(s):
   tokens = list(tokenizer.tokenize(s))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)
# df = pd.DataFrame(columns = 'label',data=[dat['tag'] for dat in gold])

Vocab size: 30522


In [11]:
def prepareData(data):
    sentence1=[]
    sentence2=[]
    paraphrased1_encode=[]
    paraphrased2_encode=[]
    for i in range(len(data)):
        dat=data[i]
        sent1=dat['sentence1']
        sent2=dat['sentence2']
        
        wordInFirst=sent1[int(dat['start1']):int(dat['end1'])]
        wordInSecond=sent2[int(dat['start2']):int(dat['end2'])]
        newSent1=sent1[0:int(dat['start1'])]+'[TGT] '+wordInFirst+' [TGT]'+sent1[int(dat['end1']):len(sent1)]
        newSent2=sent2[0:int(dat['start2'])]+'[TGT] '+ wordInSecond+' [TGT]'+sent2[int(dat['end2']):len(sent2)]
        sentence1.append(encode_sentence(newSent1))
        sentence2.append(encode_sentence(newSent2))

    return sentence1,sentence2


In [12]:
sentence1_train,sentence2_train=prepareData(data)

In [13]:
sentence1_val,sentence2_val=prepareData(dev_data)

In [14]:
sentence1_test,sentence2_test=prepareData(test_data)

In [15]:
def prepare_input(sentence1,sentence2):
    sentence1=tf.ragged.constant(sentence1)
    sentence2=tf.ragged.constant(sentence2)

    cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])] * sentence1.shape[0]
    input_word_ids = tf.concat([cls, sentence1, sentence2], axis=-1)

    input_mask = tf.ones_like(input_word_ids).to_tensor()
    type_cls = tf.zeros_like(cls)
    type_s1 = tf.ones_like(sentence1)
    type_s2 = tf.ones_like(sentence2)
    type_s2 = type_s2+1;
    input_type_ids = tf.concat([type_cls, type_s1, type_s2], axis=-1).to_tensor()
    print(type_s1.shape)
    inputs = {
        'input_word_ids': input_word_ids.to_tensor(),
        'input_mask': input_mask,
        'input_type_ids': input_type_ids}
    return inputs

In [16]:
inputsTrain=prepare_input(sentence1_train,sentence2_train)
inputsVal=prepare_input(sentence1_val,sentence2_val)
inputsTest=prepare_input(sentence1_test,sentence2_test)

(8000, None)
(1000, None)
(1000, None)


In [17]:

bert_config_file = os.path.join(gs_folder_bert, "bert_config.json")
config_dict = json.loads(tf.io.gfile.GFile(bert_config_file).read())

bert_config = bert.configs.BertConfig.from_dict(config_dict)

print(config_dict)
# config_dict={'attention_probs_dropout_prob': 0.1,
#  'hidden_act': 'relu',
#  'hidden_dropout_prob': 0.1,
#  'hidden_size': 768,
#  'initializer_range': 0.02,
#  'intermediate_size': 3072,
#  'max_position_embeddings': 512,
#  'num_attention_heads': 12,
#  'num_hidden_layers': 12,
#  'type_vocab_size': 2,
#  'vocab_size': 30522}

# bert_config = bert.configs.BertConfig.from_dict(config_dict)

# print('test')
# print(bert_config['initializer_range'])
# print(bert.bert_models.get_transformer_encoder( bert_config ,sequence_length=100))
bert_classifier, bert_encoder = bert.bert_models.classifier_model(
    bert_config, num_labels=2)
tf.keras.utils.plot_model(bert_classifier, show_shapes=True, dpi=48)
checkpoint = tf.train.Checkpoint(model=bert_encoder)
checkpoint.restore(
    os.path.join(gs_folder_bert, 'bert_model.ckpt')).assert_consumed()
# print(bert_encoder.get_embedding_table())
epochs = 6
batch_size = 32
eval_batch_size = 32

train_data_size = len(labels)
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(epochs * train_data_size * 0.1 / batch_size)

# creates an optimizer with learning rate schedule
optimizer = nlp.optimization.create_optimizer(
    2e-5, num_train_steps=num_train_steps, num_warmup_steps=warmup_steps)

metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
#                                                     patience=2,
#                                                     mode='min')
bert_classifier.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics)

bert_classifier.fit(
      inputsTrain, labels,
      validation_split=0.1,
      shuffle=True,
      batch_size=32,
      # callbacks=[early_stopping],
      epochs=epochs)
export_dir='./saved_model'
tf.saved_model.save(bert_classifier, export_dir=export_dir)

{'attention_probs_dropout_prob': 0.1, 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.1, 'hidden_size': 768, 'initializer_range': 0.02, 'intermediate_size': 3072, 'max_position_embeddings': 512, 'num_attention_heads': 12, 'num_hidden_layers': 12, 'type_vocab_size': 2, 'vocab_size': 30522}
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


INFO:absl:using Adamw optimizer


Epoch 1/6
225/225 [==============================] - 240s 1s/step - loss: 0.6585 - accuracy: 0.5861 - val_loss: 0.5892 - val_accuracy: 0.7262
Epoch 2/6
225/225 [==============================] - 236s 1s/step - loss: 0.4761 - accuracy: 0.7726 - val_loss: 0.4255 - val_accuracy: 0.8200
Epoch 3/6
225/225 [==============================] - 237s 1s/step - loss: 0.3033 - accuracy: 0.8811 - val_loss: 0.4544 - val_accuracy: 0.8263
Epoch 4/6
225/225 [==============================] - 237s 1s/step - loss: 0.1757 - accuracy: 0.9425 - val_loss: 0.5467 - val_accuracy: 0.8213
Epoch 5/6
225/225 [==============================] - 238s 1s/step - loss: 0.1080 - accuracy: 0.9661 - val_loss: 0.6938 - val_accuracy: 0.8225
Epoch 6/6
225/225 [==============================] - 237s 1s/step - loss: 0.0812 - accuracy: 0.9761 - val_loss: 0.7555 - val_accuracy: 0.8150



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)

FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: ./saved_model/assets


INFO:tensorflow:Assets written to: ./saved_model/assets


In [ ]:
bert_classifier_model = tf.keras.models.load_model('drive/MyDrive/saved_model/')


In [18]:
from sklearn.metrics import classification_report

In [19]:
from sklearn.metrics import accuracy_score


In [20]:
result= bert_classifier.predict(inputsTest)
result = tf.argmax(result,axis=1).numpy()
# print(test_labels.shape)
print(accuracy_score(labels_test,result))

0.855
